Chatbot by Likhit Dadhey

In [1]:

import pandas as pd
import nltk
import re
from nltk.stem import wordnet #to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag #for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words


In [2]:

df=pd.read_excel('dialog_talk_agent.xlsx')
df.head(20)

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,NaN
3,Describe yourself,NaN
4,tell me about yourself,NaN
5,all about you,NaN
6,tell me some stuff about you,NaN
7,talk some stuff about you,NaN
8,talk about yourself,NaN
9,about yourself,NaN


In [3]:
df.shape[0] # returns the number of rows in dataset

1592

In [4]:
df.ffill(axis = 0,inplace=True) # fills the null value with the previous value.
df

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder
...,...,...
1587,can we chat,Talking is what I do best.
1588,I'll be back in a few minutes,I'll be waiting.
1589,I'll be back,All right. I'll be here.
1590,I'll get back to you in a moment,Till next time.


In [5]:
df1=df.head(10) # copy of first ten rows of dataset

In [6]:
# function that converts text into lower case and removes special characters

def step1(x):
    for i in x:
        a=str(i).lower()
        p=re.sub(r'[^a-z0-9]',' ',a)
        print(p)

In [7]:
step1(df1['Context'])

tell me about your personality
i want to know you better
define yourself
describe yourself
tell me about yourself
all about you
tell me some stuff about you
talk some stuff about you
talk about yourself
about yourself


In [8]:
 # word tokenizing
    
s='tell me about your personality'
words=word_tokenize(s)
print(words)

['tell', 'me', 'about', 'your', 'personality']


In [9]:
lemma = wordnet.WordNetLemmatizer() # intializing lemmatizer
lemma.lemmatize('absorbed', pos = 'v')

'absorb'

In [10]:
pos_tag(nltk.word_tokenize(s),tagset = None) # returns the parts of speech of every word


[('tell', 'VB'),
 ('me', 'PRP'),
 ('about', 'IN'),
 ('your', 'PRP$'),
 ('personality', 'NN')]

In [11]:

# function that performs text normalization steps

def text_normalization(text):
    text=str(text).lower() # text to lower case
    spl_char_text=re.sub(r'[^ a-z]','',text) # removing special characters
    tokens=nltk.word_tokenize(spl_char_text) # word tokenizing
    lema=wordnet.WordNetLemmatizer() # intializing lemmatization
    tags_list=pos_tag(tokens,tagset=None) # parts of speech
    lema_words=[]   # empty list 
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):  # Verb
            pos_val='v'
        elif pos_token.startswith('J'): # Adjective
            pos_val='a'
        elif pos_token.startswith('R'): # Adverb
            pos_val='r'
        else:
            pos_val='n' # Noun
        lema_token=lema.lemmatize(token,pos_val) # performing lemmatization
        lema_words.append(lema_token) # appending the lemmatized token into a list
    
    return " ".join(lema_words) # returns the lemmatized tokens as a sentence 

In [12]:
text_normalization('telling you some stuff about me')

'tell you some stuff about me'

In [13]:
df['lemmatized_text']=df['Context'].apply(text_normalization) # applying the fuction to the dataset to get clean text
df.tail(15)

,Context,Text Response,lemmatized_text
1577,I need to talk to you,Good conversation really makes my day.,i need to talk to you
1578,I want to speak with you,I'm always here to lend an ear.,i want to speak with you
1579,let's have a discussion,Talking is what I do best.,let have a discussion
1580,I just want to talk,Talking is what I do best.,i just want to talk
1581,let's discuss something,Talking is what I do best.,let discuss something
1582,can I speak,Talking is what I do best.,can i speak
1583,can we talk,Talking is what I do best.,can we talk
1584,let's talk,Talking is what I do best.,let talk
1585,I want to talk to you,Talking is what I do best.,i want to talk to you
1586,can we chat,Talking is what I do best.,can we chat


In [14]:
# all the stop words we have 

stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Bag of words

In [15]:
cv = CountVectorizer() # intializing the count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [16]:
# returns all the unique word from data 

features = cv.get_feature_names()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,afraid,...,yeh,yep,yes,yet,you,your,youre,yours,yourself,yup
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
Question ='Will you help me and tell me about yourself more' # considering an example query


In [18]:
# checking for stop words

Q=[]
a=Question.split()
for i in a:
    if i in stop:
        continue
    else:
        Q.append(i)
    b=" ".join(Q)

In [19]:
Question_lemma = text_normalization(b) # applying the function that we created for text normalizing
Question_bow = cv.transform([Question_lemma]).toarray() # applying bow

In [20]:
text_normalization


<function __main__.text_normalization(text)>

In [21]:
Question_bow

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

similarity

In [22]:
# cosine similarity for the above question we considered.

cosine_value = 1- pairwise_distances(df_bow, Question_bow, metric = 'cosine' )
(cosine_value)

array([[0.25819889],
       [0.        ],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [23]:
df['similarity_bow']=cosine_value # creating a new column 

In [24]:

df['similarity_bow']=cosine_value # creating a new column 
df_simi = pd.DataFrame(df, columns=['Text Response','similarity_bow']) 
# taking similarity value of responses for the question we took
df_simi


,Text Response,similarity_bow
0,Just think of me as the ace up your sleeve.,0.258199
1,I can help you work smarter instead of harder,0.000000
2,I can help you work smarter instead of harder,0.000000
3,I can help you work smarter instead of harder,0.000000
4,I can help you work smarter instead of harder,0.288675
...,...,...
1587,Talking is what I do best.,0.000000
1588,I'll be waiting.,0.000000
1589,All right. I'll be here.,0.000000
1590,Till next time.,0.000000


In [25]:
df_simi_sort = df_simi.sort_values(by='similarity_bow', ascending=False) # sorting the values
df_simi_sort.head()

,Text Response,similarity_bow
211,I'm glad to help. What can I do for you?,0.577350
194,I'm glad to help. What can I do for you?,0.577350
184,I'm glad to help. What can I do for you?,0.408248
186,I'm glad to help. What can I do for you?,0.408248
200,I'm glad to help. What can I do for you?,0.408248


In [26]:
threshold = 0.2 # considering the value of p=smiliarity to be greater than 0.2
df_threshold = df_simi_sort[df_simi_sort['similarity_bow'] > threshold] 
df_threshold

,Text Response,similarity_bow
211,I'm glad to help. What can I do for you?,0.577350
194,I'm glad to help. What can I do for you?,0.577350
184,I'm glad to help. What can I do for you?,0.408248
186,I'm glad to help. What can I do for you?,0.408248
200,I'm glad to help. What can I do for you?,0.408248
219,I'm glad to help. What can I do for you?,0.333333
728,It's my pleasure to help.,0.333333
188,I'm glad to help. What can I do for you?,0.333333
190,I'm glad to help. What can I do for you?,0.333333
191,I'm glad to help. What can I do for you?,0.333333


* Finally using bow for the question 'Will you help me and tell me about yourself more' , the above are the responses we got using bow and the smiliarity value of responses, we consider the response with highest similarity

In [27]:
index_value = cosine_value.argmax() # returns the index number of highest value
index_value 

194

In [28]:
(Question)

'Will you help me and tell me about yourself more'

In [29]:
df['Text Response'].loc[index_value] # The text at the above index becomes the response for the question


"I'm glad to help. What can I do for you?"

tf-idf

In [30]:
Question1 ='Tell me about yourself.'

In [31]:
tfidf=TfidfVectorizer() # intializing tf-id 
x_tfidf=tfidf.fit_transform(df['lemmatized_text']).toarray()

In [32]:
df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names()) 
df_tfidf.head()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,afraid,...,yeh,yep,yes,yet,you,your,youre,yours,yourself,yup
0,0.0,0.407572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.330555,0.0,0.0,0.000000,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.218768,0.000000,0.0,0.0,0.000000,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.641790,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.641790,0.0
4,0.0,0.453790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.608937,0.0


In [33]:
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.transform([Question_lemma]).toarray() # applying tf-idf

Similarity 


In [34]:
cos=1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')  # applying cosine similarity
cos

array([[0.28640647],
       [0.        ],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [35]:
df['similarity_tfidf']=cos # creating a new column 
df_simi_tfidf = pd.DataFrame(df, columns=['Text Response','similarity_tfidf']) # taking similarity value of responses for the question we took
df_simi_tfidf 

,Text Response,similarity_tfidf
0,Just think of me as the ace up your sleeve.,0.286406
1,I can help you work smarter instead of harder,0.000000
2,I can help you work smarter instead of harder,0.000000
3,I can help you work smarter instead of harder,0.000000
4,I can help you work smarter instead of harder,0.318885
...,...,...
1587,Talking is what I do best.,0.000000
1588,I'll be waiting.,0.000000
1589,All right. I'll be here.,0.000000
1590,Till next time.,0.000000


In [36]:
df_simi_tfidf_sort = df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False) # sorting the values
df_simi_tfidf_sort.head(10)

,Text Response,similarity_tfidf
194,I'm glad to help. What can I do for you?,0.712435
211,I'm glad to help. What can I do for you?,0.490298
288,"Oh, don't give up on me!",0.452165
414,Absolutely. You don't have to ask.,0.424411
538,My pleasure.,0.422017
16,I can help you work smarter instead of harder,0.391137
184,I'm glad to help. What can I do for you?,0.384952
419,Absolutely. You don't have to ask.,0.366138
214,I'm glad to help. What can I do for you?,0.366006
733,It's my pleasure to help.,0.363487


In [37]:
threshold = 0.2 # considering the value of p=smiliarity to be greater than 0.2
df_threshold = df_simi_tfidf_sort[df_simi_tfidf_sort['similarity_tfidf'] > threshold] 
df_threshold

,Text Response,similarity_tfidf
194,I'm glad to help. What can I do for you?,0.712435
211,I'm glad to help. What can I do for you?,0.490298
288,"Oh, don't give up on me!",0.452165
414,Absolutely. You don't have to ask.,0.424411
538,My pleasure.,0.422017
16,I can help you work smarter instead of harder,0.391137
184,I'm glad to help. What can I do for you?,0.384952
419,Absolutely. You don't have to ask.,0.366138
214,I'm glad to help. What can I do for you?,0.366006
733,It's my pleasure to help.,0.363487


* by using tfidf for the question 'Will you help me and tell me about yourself more' , the above are the responses we got and the smiliarity value of responses, we consider the response with highest similarity

In [38]:
index_value1 = cos.argmax() # returns the index number of highest value
index_value1

194

In [39]:
Question1

'Tell me about yourself.'

In [40]:
df['Text Response'].loc[index_value1]  # returns the text at that index

"I'm glad to help. What can I do for you?"

Model Using Bag of Words

In [41]:
# Function that removes stop words and process the text

def stopword_(text):   
    tag_list=pos_tag(nltk.word_tokenize(text),tagset=None)
    stop=stopwords.words('english')
    lema=wordnet.WordNetLemmatizer()
    lema_word=[]
    for token,pos_token in tag_list:
        if token in stop:
            continue
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)
        lema_word.append(lema_token)
    return " ".join(lema_word)

In [42]:
# defining a function that returns response to query using bow

def chat_bow(text):
    s=stopword_(text)
    lemma=text_normalization(s) # calling the function to perform text normalization
    bow=cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value=cosine_value.argmax() # getting index value 
    return df['Text Response'].loc[index_value]

In [43]:
chat_bow('hi there')

'Hey!'

In [44]:
chat_bow('Your are amazing')

'Terrific!'

In [45]:
chat_bow('i miss you')

"I've been right here all along!"

Model Using tf-idf

In [46]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma=text_normalization(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df['Text Response'].loc[index_value]

In [47]:
chat_tfidf('hi')

'Hey!'

In [48]:
chat_tfidf('how are you')

'Lovely, thanks.'

In [49]:
chat_tfidf('i love you')

"That's great to hear."

In [50]:
chat_tfidf('thanks for your support!')

"It's my pleasure to help."

In [51]:
chat_tfidf('will you reply accurately?')

"Oh, don't give up on me!"

In [52]:
chat_tfidf('will you marry me?')

'In the virtual sense that I can, sure.'

In [53]:
chat_tfidf('i miss you and i love you')

"I've been right here all along!"

In [54]:
chat_tfidf('ask sravya to read')

"Oops. Sorry about that. I'm still learning."

In [55]:
chat_tfidf('you are amazing and hope to see u soon.')

'Bye.'

# Conclusion

Our chat bot worked well with both bow and tf-idf, tf-idf model worked well even with stop words compared to bow where we had to remove stop words before using bag of words.